## SQLite
***

SQLite é o banco de dados open source mais simples, usando bastente no ambiente de desenvolvimento e teste, mas não é aconselhado para ambientes de produção.

O SQLite é uma biblioteca C que fornece um banco de dados leve baseado em disco que não requer um processo de servidor separado e permite acessar o banco de dados usando uma variante não padrão da linguagem de consulta SQL. Alguns aplicativos podem usar o SQLite para armazenamento interno de dados. Também é possível prototipar um aplicativo usando o SQLite e depois portar o código para um banco de dados maior, como o PostgreSQL ou o Oracle.

Instalação:

```
sudo apt-get install sqlite3
```

Para executar um comando no sqlite:

```
sqlite3 <banco-de-dados>.db 'comando'
```

Comandos mais comuns:

```
.tables - Lista de tabelas
.schema tabela - Visualiza o código SQL que criou as tabelas
.exit - Sai do banco de dados
.header on - Mostra o nome das colunas ao executar o SELECT
.output arquivo.txt - Escreve o resultados em um arquivo externo
.mode column - Visualização mais agradavel das colunas
PRAGMA table_info(tabela) - Retorna os campos da tabela
```

Backup e Restauração:

```
sqlite3 banco.db .dump > script.sql - Exporta os comandos SQL do banco de dados
sqlite3 banco.db < script.sql - Importa os comandos SQL no banco de dados
```

Verificar versão do SQLite:

```
SELECT SQLITE_VERSION()
```

Renomear tabela:

```
ALTER TABLE clientes RENAME TO funcionarios;
```

Erros:

```
Maioria dos erros (conexão, schema, ...): sqlite3.Error
Inserir registros (commit): sqlite3.IntegrityError
```

***
### Conexão
***

In [1]:
import sqlite3

In [2]:
# Connection é o objeto que representa o banco de dados
# Se quisermos usar na memória é só passar ':memory:'
conexao = sqlite3.connect('./sqlite.db')

In [3]:
# Vamos criar um cursor, ou seja, um iterador que permite navegar e manipular os registros do DB.
cursor = conexao.cursor()

***
### Criando uma tabela
***

In [4]:
# Criar uma tabela
cursor.execute("""
CREATE TABLE IF NOT EXISTS clientes (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    cpf VARCHAR(11) NOT NULL,
    email TEXT NOT NULL,
    cidade_id INTEGER,
    criado_em DATE NOT NULL,
    FOREIGN KEY (cidade_id) REFERENCES cidades(id)
);
""")

In [5]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS cidades (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    cidade TEXT,
    uf VARCHAR(2)
);
""")

***
### Inserindo registros
***

In [6]:
cidades = [('Campinas','SP'),
           ('Sao Paulo','SP'),
           ('Rio de Janeiro','RJ')]

In [7]:
cursor.executemany("""
INSERT INTO cidades (cidade, uf) VALUES (?,?)
""", cidades)

In [8]:
clientes = [('Regis', '01119239423', 'regis@gmail.com', 1, '2014-06-08'),
            ('Aloisio', '02320322932', 'aloisio@email.com', 2, '2014-06-09'),
            ('Bruna', '11239428344', 'bruna@email.com', 2, '2014-06-09'),
            ('Matheus', '01293844323', 'matheus@email.com', 3, '2014-06-08')]

In [9]:
cursor.executemany("""
INSERT INTO clientes (nome, cpf, email, cidade_id, criado_em) VALUES (?,?,?,?,?)
""", clientes)

In [10]:
# Salvando os dados no BD.
conexao.commit()

***
### Lendo os dados
***

In [11]:
# Registros de clientes
cursor.execute("""
SELECT * FROM clientes;
""")

In [12]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis@gmail.com', 1, '2014-06-08')
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, '2014-06-09')
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, '2014-06-09')
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, '2014-06-08')


In [13]:
# Registros de cidades
cursor.execute("""
SELECT * FROM cidades;
""")

In [14]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Campinas', 'SP')
(2, 'Sao Paulo', 'SP')
(3, 'Rio de Janeiro', 'RJ')


In [15]:
# Registros de clientes e cidades
cursor.execute("""
SELECT * FROM clientes INNER JOIN cidades ON clientes.cidade_id = cidades.id;
""")

In [16]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis@gmail.com', 1, '2014-06-08', 1, 'Campinas', 'SP')
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, '2014-06-09', 2, 'Sao Paulo', 'SP')
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, '2014-06-09', 2, 'Sao Paulo', 'SP')
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, '2014-06-08', 3, 'Rio de Janeiro', 'RJ')


***
### Alterando os dados
***

In [17]:
id_cliente = 1
novo_email = 'regis_novo@gmail.com'
novo_criado_em = '2014-06-11'

In [18]:
cursor.execute("""
UPDATE clientes
SET email = ?, criado_em = ?
WHERE id = ?
""", (novo_email, novo_criado_em, id_cliente))

In [19]:
conexao.commit()

In [20]:
cursor.execute("""
SELECT * FROM clientes;
""")

In [21]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis_novo@gmail.com', 1, '2014-06-11')
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, '2014-06-09')
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, '2014-06-09')
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, '2014-06-08')


***
### Deletando um registro
***

In [22]:
id_cliente = 4

In [23]:
cursor.execute("""
DELETE FROM clientes
WHERE id = ?
""", (id_cliente,))

In [24]:
conexao.commit()

In [25]:
cursor.execute("""
SELECT * FROM clientes;
""")

In [26]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis_novo@gmail.com', 1, '2014-06-11')
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, '2014-06-09')
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, '2014-06-09')


***
### Adicionar uma nova coluna
***

In [27]:
cursor.execute("""
ALTER TABLE clientes
ADD COLUMN bloqueado BOOLEAN;
""")

In [28]:
conexao.commit()

***
### Lendo informações do banco de dados
***

In [29]:
tabela = 'clientes'

In [30]:
# Obtendo informações da tabela
cursor.execute('PRAGMA table_info({})'.format(tabela))
colunas = [tupla[1] for tupla in cursor.fetchall()]
print('Colunas:', colunas)

Colunas: ['id', 'nome', 'cpf', 'email', 'cidade_id', 'criado_em', 'bloqueado']


In [31]:
# Listando as tabelas do BD
cursor.execute("""
SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
""")
print('Tabelas:')
for tabela in cursor.fetchall():
    print("%s" % (tabela))

Tabelas:
cidades
clientes
sqlite_sequence


In [32]:
# Obtendo o schema da tabela
cursor.execute("""
SELECT sql FROM sqlite_master WHERE type='table' AND name='clientes'
""")
print("Schema:")
for schema in cursor.fetchall():
    print("%s" % (schema))

Schema:
CREATE TABLE clientes (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    cpf VARCHAR(11) NOT NULL,
    email TEXT NOT NULL,
    cidade_id INTEGER,
    criado_em DATE NOT NULL, bloqueado BOOLEAN,
    FOREIGN KEY (cidade_id) REFERENCES cidades(id)
)


***
### Fazendo backup do banco de dados (exportando)
***

In [33]:
import io

In [34]:
with io.open('clientes_dump.sql', 'w') as file:
    for linha in conexao.iterdump():
        file.write('%s\n' % linha)

***
### Deletando tabela
***

In [35]:
# Deleta todas as instâncias
cursor.execute("DELETE FROM clientes")
cursor.execute("DELETE FROM cidades")

In [36]:
cursor.execute("DROP TABLE clientes;")
cursor.execute("DROP TABLE cidades;")
conexao.commit()

***
### Recuperando o banco de dados
***

In [37]:
file = io.open('clientes_dump.sql', 'r')

In [38]:
sql = file.read()

In [39]:
cursor.executescript(sql)

In [40]:
cursor.execute("""
SELECT * FROM clientes;
""")

In [41]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis_novo@gmail.com', 1, '2014-06-11', None)
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, '2014-06-09', None)
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, '2014-06-09', None)


In [42]:
# Fechando a conexão
conexao.close()